### Import python library

In [1]:
import numpy as np
import pandas as pd

### 1. Read data: augment_24group_1620.csv필요

In [2]:
data = pd.read_csv("bigcon_final_dataset.csv")

In [3]:
data.drop(["Unnamed: 0"],axis=1, inplace = True)

In [4]:
team = list(set(data.T_ID))

In [5]:
def shift_data(sel_col,df, shift_col):
    df_shift = df.loc[:,sel_col]
    
    for i in range(1,3):
        for c in shift_col:
            df_shift.loc[:,'shift_{}_{}'.format(c,i)] = df.loc[:,c].shift(i)
            
            
    df_shift.dropna(inplace = True)
    df_idx = list(df_shift.index)

    drop_idx = []
    for i in df_idx:
        idx_num = df_shift.loc[i,'IDX']
        if idx_num <= df_shift.loc[i,'shift_IDX_1'] or idx_num <= df_shift.loc[i,'shift_IDX_2']  :
            drop_idx.append(i)

    df_s_df = df_shift.drop(drop_idx).reset_index(drop = True)
    
    return df_s_df

### 2. Train 전처리

In [6]:
train = data[data["IDX"]!=777]

sel_col = ["T_ID","IDX","MERGE_IDX","PCT","ERA","AVG",'YEAR']
shift_col = sorted(list(set(list(train.columns))-set(['T_ID',"YEAR"])))

train_s_df = shift_data(sel_col, train, shift_col)
train_s_drop_df = train_s_df.drop(["IDX","MERGE_IDX","shift_IDX_1","shift_MERGE_IDX_1",
                                   "shift_IDX_2","shift_MERGE_IDX_2","ERA","AVG","PCT"],axis = 1)

In [7]:
train_s_df.shape

(1960, 125)

### 3. Test 전처리

In [8]:
test = data[data["IDX"]==777]

In [9]:
tmp5 = data[(data["IDX"]==5)]
tmp4 = data[(data["IDX"]==4) & (data["MERGE_IDX"]==4)]

test = pd.concat([test,tmp5, tmp4],axis=0)

test = test.sort_values(by=["YEAR","T_ID","IDX"]).reset_index(drop=True)

In [10]:
shift_col = sorted(list(set(list(test.columns))-set(['T_ID','YEAR'])))

test_s_df = shift_data(sel_col, test, shift_col)

test_s_df = test_s_df[test_s_df["IDX"]==777].reset_index(drop = True)
test_s_drop_df = test_s_df.drop(["IDX","MERGE_IDX","shift_IDX_1","shift_MERGE_IDX_1",
                                   "shift_IDX_2","shift_MERGE_IDX_2","ERA","AVG","PCT"],axis = 1)

In [11]:
PCT_lstm_train_X = train_s_drop_df.reset_index(drop=True)
PCT_lstm_train_y =  train_s_df[["T_ID","YEAR","PCT"]]

PCT_lstm_test_X = test_s_drop_df.reset_index(drop=True)
PCT_lstm_test_y = test_s_df[["T_ID","YEAR","PCT"]]

In [27]:
PCT_lstm_train_X.to_csv("PCT_lstm_final_train_X.csv",index=False)
PCT_lstm_train_y.to_csv("PCT_lstm_final_train_y.csv",index=False)

PCT_lstm_test_X.to_csv("PCT_lstm_final_test_X.csv",index=False)
PCT_lstm_test_y.to_csv("PCT_lstm_final_test_y.csv",index=False)

In [12]:
# shift 추가
common_list = ['T_ID', 'YEAR', 'HEADER_NO','PCT']
hitter = ['AVG', 'OBP', 'SLG', 'OPS', 'RC', 'GPA', 'BA', 'SECA', 
          'TA', 'XR', 'ISO', 'wOBA', 'P_HRA_RT', 'DER']
pitcher = ['WHIP', 'LOB', 'ERA', 'FIP', 'H_9', 'K_9', 'BB_9', 
           'HR_9', 'oAVG', 'oSLG', 'oOBP', 'P_WHIP_RT', 'P2_WHIP_RT', 'CB_WHIP_RT','oISO', 'oOPS']

vs_hitter = ['vs_AVG', 'vs_OBP', 'vs_SLG', 'vs_OPS', 'vs_RC', 'vs_GPA', 'vs_BA', 'vs_SECA', 
             'vs_TA', 'vs_XR', 'vs_ISO', 'vs_wOBA', 'vs_P_HRA_RT', 'vs_DER']
vs_pitcher = ['vs_WHIP', 'vs_LOB', 'vs_ERA', 'vs_FIP', 'vs_H_9', 'vs_K_9', 'vs_BB_9', 
              'vs_HR_9', 'vs_oAVG', 'vs_oSLG', 'vs_oOBP', 'vs_P_WHIP_RT', 'vs_P2_WHIP_RT', 'vs_CB_WHIP_RT'
             ,'vs_oISO', 'vs_oOPS']

avg_col = common_list + hitter + vs_pitcher
era_col = common_list + pitcher + vs_hitter

# s_hit = add_shift(hitter)
# s_pit = add_shift(pitcher)
# s_vs_hit = add_shift(vs_hitter)
# s_vs_pit = add_shift(vs_pitcher)

In [13]:
train_col = list(train_s_drop_df.columns)
sel_avg_lst =train_col[:2]
sel_era_lst = train_col[:2]

for i in train_col[2:]:
    for k in avg_col:
        if k in i:
            sel_avg_lst.append(i)
            break
    
for i in train_col[2:]:
    for k in era_col:
        if k in i:
            sel_era_lst.append(i)
            break


In [57]:
AVG_lstm_train_X = train_s_drop_df.reset_index(drop=True)
AVG_lstm_train_y =  train_s_df[["T_ID","YEAR","AVG"]]

AVG_lstm_test_X = test_s_drop_df.reset_index(drop=True)
AVG_lstm_test_y = test_s_df[["T_ID","YEAR","AVG"]]

In [58]:
AVG_lstm_train_X_hit = AVG_lstm_train_X[sel_avg_lst]
AVG_lstm_test_X_hit = AVG_lstm_test_X[sel_avg_lst]

In [59]:
AVG_lstm_train_X_hit.to_csv("AVG_lstm_final_train_X_hit.csv",index=False)
AVG_lstm_train_y.to_csv("AVG_lstm_final_train_y_hit.csv",index=False)

AVG_lstm_test_X_hit.to_csv("AVG_lstm_final_test_X_hit.csv",index=False)
AVG_lstm_test_y.to_csv("AVG_lstm_final_test_y_hit.csv",index=False)

In [14]:
# 방어율
ERA_train_X = train_s_drop_df[sel_era_lst]
ERA_train_y = train_s_df[["T_ID","YEAR","ERA"]]

ERA_test_X = test_s_drop_df[sel_era_lst]
ERA_test_y = test_s_df[["T_ID","YEAR","ERA"]]

In [15]:
ERA_train_X.to_csv("ERA_lstm_final_train_X.csv",index=False)
ERA_train_y.to_csv("ERA_lstm_final_train_y.csv",index=False)

ERA_test_X.to_csv("ERA_lstm_final_test_X.csv",index=False)
ERA_test_y.to_csv("ERA_lstm_final_test_y.csv",index=False)